In [ ]:
import h5py
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import KFold

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
TRAIN_PATH = r'.\data\train.mp4'
LABEL_PATH = r'.\data\train.txt'
H5PY_PATH  = r'.\data\train.h5'

In [ ]:
class DataGenerator(keras.utils.Sequence):

    def __init__(
        self, data_path, indexes=None, n_channels=3, batch_size=16, shuffle=True
    ):
        self.batch_size = batch_size
        self.data_path = data_path
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.file = h5py.File(data_path, 'r')
        if indexes is None:
            self.indexes = np.arange(len(self.file['OPF']))
        else:
            self.indexes = indexes

    def __len__(self):
        return int(np.floor(len(self.indexes) / self.batch_size))

    def __getitem__(self, index):
        X, y = self.__data_generation(
            list(self.indexes[index*self.batch_size:(index+1)*self.batch_size])
        )
        return X, y

    def __del__(self):
        self.file.close()

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, index):
        index = list(index)
        index.sort()
        X = np.array(self.file['OPF'][index])
        y = np.array(self.file['LBL'][index])
        return X, y

In [ ]:
class SpeedNet(keras.Model):

    def __init__(self, input_dim=(150, 300, 2), kernel_size=3):
        super(SpeedNet, self).__init__()
        self.layer0 = Conv2D(8, (kernel_size, kernel_size), activation='relu', data_format='channels_last')
        self.layer1 = MaxPooling2D(pool_size=(1, 2))
        self.layer2 = Conv2D(16, (kernel_size, kernel_size), activation='relu', data_format='channels_last')
        self.layer3 = MaxPooling2D(pool_size=(1, 2))
        self.layer4 = Conv2D(32, (kernel_size, kernel_size), activation='relu', data_format='channels_last')
        self.layer5 = MaxPooling2D()
        self.layer6 = Conv2D(64, (kernel_size, kernel_size), activation='relu', data_format='channels_last')
        self.layer7 = MaxPooling2D()
        self.layer8 = Conv2D(128, (kernel_size, kernel_size), activation='relu', data_format='channels_last')
        self.layer9 = MaxPooling2D()
        self.layer10 = Conv2D(256, (kernel_size, kernel_size), activation='relu', data_format='channels_last')
        self.layer11 = MaxPooling2D()
        self.layer12 = Conv2D(512, (kernel_size, kernel_size), activation='relu', data_format='channels_last', padding='same')
        self.layer13 = MaxPooling2D()
        self.layer14 = GlobalAveragePooling2D()
        self.layer15 = Dropout(0.2)
        self.layer16 = Dense(500, activation='relu')
        self.layer17 = Dense(250, activation='relu')
        self.layer18 = Dense(1, activation='relu')

    def call(self, x):
        x = self.layer0(x)
        print(x.shape)
        x = self.layer1(x)
        print(x.shape)
        x = self.layer2(x)
        print(x.shape)
        x = self.layer3(x)
        print(x.shape)
        x = self.layer4(x)
        print(x.shape)
        x = self.layer5(x)
        print(x.shape)
        x = self.layer6(x)
        print(x.shape)
        x = self.layer7(x)
        print(x.shape)
        x = self.layer8(x)
        print(x.shape)
        x = self.layer9(x)
        print(x.shape)
        x = self.layer10(x)
        print(x.shape)
        x = self.layer12(x)
        print(x.shape)
        x = self.layer13(x)
        print(x.shape)
        x = self.layer14(x)
        print(x.shape)
        x = self.layer15(x)
        print(x.shape)
        x = self.layer16(x)
        print(x.shape)
        x = self.layer17(x)
        print(x.shape)
        x = self.layer18(x)
        print(x.shape)
        return x

In [ ]:
TRAIN_COUNT = 20399

kf = KFold(n_splits=5)

valid_loss = []
fold_var = 0

for train_index, val_index in kf.split(np.arange(TRAIN_COUNT)):
    train_data_generator = DataGenerator(
        H5PY_PATH, indexes=train_index, batch_size=32
    )
    valid_data_generator = DataGenerator(
        H5PY_PATH, indexes=val_index, batch_size=32
    )

    model = SpeedNet()
    model.compile(loss='mse', optimizer='adam')

    es = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1,
        patience=3
    )
    mc = keras.callbacks.ModelCheckpoint(
        f'./models/best_model_{fold_var}.h5',
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )

    history = model.fit(
        train_data_generator,
        epochs=100,
        callbacks=[es, mc],
        validation_data=valid_data_generator
    )

    model.load_weights(f'./models/best_model_{fold_var}.h5')

    results = model.evaluate(valid_data_generator)
    valid_loss.append(results)

    keras.backend.clear_session()

    fold_var += 1